# Single Table Modeling

**SDV** supports modeling single table datasets. It provides unique features for making it easy for the user 
to learn models and synthesize datasets. Some important features of sdv.tables include:

* Support for tables with primarykey
* Support to anonymize certain fields like addresses, emails, phone numbers, names and other PII information. 
  We use faker library for this. The full list of categories supported corresponds to the `Faker` library 
  [provider names](https://faker.readthedocs.io/en/master/providers.html).
* Support for a number of different data types - categorical, numerical, discrete-ordinal and datetimes.
* Support multiple types of statistical and deep learning models:
  * GaussianCopula: A tool to model multivariate distributions using [copula functions](
    https://en.wikipedia.org/wiki/Copula_%28probability_theory%29). Based on our [Copulas Library](
    https://github.com/sdv-dev/Copulas).
  * CTGAN: A GAN-based Deep Learning data synthesizer that can generate synthetic tabular data with high 
    fidelity. Based on our [CTGAN Library](https://github.com/sdv-dev/CTGAN).

**Note:** We are adding a number of additional features and functionality to make it easy to model single table datasets. For example, we are adding ways for users to add inter-column constraints . If you find a unique use case that we do not support consider suggesting and adding examples here.

## Quick usage

Let's consider a dataset from our demo datasets. 

In [1]:
# Setup logging and warnings
import logging;
logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(level=logging.WARNING)
logging.getLogger('sdv').setLevel(level=logging.INFO)

import warnings
warnings.simplefilter("ignore")

In [2]:
from sdv import load_demo

users = load_demo()['users']

This will return a table with 4 fields:

* `user_id`: A unique identifier of the user.
* `country`: A 2 letter code of the country of residence of the user.
* `gender`: A single letter code, `M` or `F`, indicating the user gender. Note that this demo simulates the case where some users did not indicate the gender, which resulted in empty data values in some rows.
* `age`: The age of the user, in years.

In [3]:
users

,user_id,country,gender,age
0,0,US,M,34
1,1,UK,F,23
2,2,ES,None,44
3,3,UK,M,22
4,4,US,F,54
5,5,DE,M,57
6,6,BG,F,45
7,7,ES,None,41
8,8,FR,F,23
9,9,UK,None,30


We notice that there are some additional properties in this dataset:

* First, `user_id` field in our table is the `primary_key` and each row has a `unique` value, so we do not
  want our model to attempt to learn it.
* Second, let's say we want to `anonymize` the countries of residence of our `users`, to avoid disclosing
  such information. 
* Third, we notice that there is missing data for the `gender` column. 

Let us use the `GaussianCopula` to model this data and then sample synthetic data from the model. In order
to properly model our data we will need to provide some additional information to our model. Once we have
prepared the arguments for our model we are ready to import it, create an instance and fit it to our data.

In [4]:
from sdv.tabular import GaussianCopula

model = GaussianCopula(
    primary_key='user_id',
    anonymize_fields={'country':'country_code'}
)
model.fit(users)

2020-08-05 10:37:05,854 - INFO - sdv.metadata.table - Loading transformer OneHotEncodingTransformer for field country
2020-08-05 10:37:05,855 - INFO - sdv.metadata.table - Loading transformer OneHotEncodingTransformer for field gender
2020-08-05 10:37:05,855 - INFO - sdv.metadata.table - Loading transformer NumericalTransformer for field age


**Notice** that the model `fitting` process took care of transforming the different fields using the
appropriate [Reversible Data Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has
a format that the GaussianMultivariate model from the [copulas](https://github.com/sdv-dev/Copulas)
library can handle.

## Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from your model.

In [5]:
sampled = model.sample(5)

This will return a table identical to the one which the model was fitted on, but filled with new data
which resembles the original one.

In [6]:
sampled

,user_id,country,gender,age
0,0,GB,F,19
1,1,BN,M,52
2,2,GB,F,18
3,3,RU,F,43
4,4,GB,F,24


**Note:** You can control the number of rows by specifying the number of `samples` in the
`model.sample(<num_rows>)`. To test, try `model.sample(10000)`. Note that the original 
table only had 10 rows.

## Let's consider using CTGAN

In this second part of the tutorial we will be using the CTGAN model to learn the data from the
demo dataset called `census`, which is based on the [UCI Adult Census Dataset]('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data').

In [7]:
from sdv import load_demo

census = load_demo('census')['census']

2020-08-05 10:37:07,440 - INFO - sdv.metadata - Loading table census


This will return a table with several rows of multiple data types:

In [8]:
census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In this case there is no `primary_key` to setup and we will not be `anonymizing` anything, so the
only thing that we will pass to the `CTGAN` model is the `number of epochs` that we want it to
perform when it learns the data, which we will keep low to make this execution quick.

In [9]:
from sdv.tabular import CTGAN

model = CTGAN(epochs=10)

Once the instance is created, we can fit it to our data. 

**Note** that this process might take some time to finish, especially on non-GPU enabled systems,
so in this case we will be passing only a `subsample` of the data to accelerate the process.

In [10]:
model.fit(census.sample(1000))

2020-08-05 10:37:07,997 - INFO - sdv.metadata.table - Loading transformer NumericalTransformer for field age
2020-08-05 10:37:07,998 - INFO - sdv.metadata.table - Loading transformer LabelEncodingTransformer for field workclass
2020-08-05 10:37:07,998 - INFO - sdv.metadata.table - Loading transformer NumericalTransformer for field fnlwgt
2020-08-05 10:37:07,999 - INFO - sdv.metadata.table - Loading transformer LabelEncodingTransformer for field education
2020-08-05 10:37:07,999 - INFO - sdv.metadata.table - Loading transformer NumericalTransformer for field education-num
2020-08-05 10:37:08,000 - INFO - sdv.metadata.table - Loading transformer LabelEncodingTransformer for field marital-status
2020-08-05 10:37:08,000 - INFO - sdv.metadata.table - Loading transformer LabelEncodingTransformer for field occupation
2020-08-05 10:37:08,000 - INFO - sdv.metadata.table - Loading transformer LabelEncodingTransformer for field relationship
2020-08-05 10:37:08,001 - INFO - sdv.metadata.table - Lo

Epoch 1, Loss G: 1.9563, Loss D: -0.0111
Epoch 2, Loss G: 1.9205, Loss D: -0.0590
Epoch 3, Loss G: 1.9457, Loss D: -0.1371
Epoch 4, Loss G: 1.9826, Loss D: -0.2043
Epoch 5, Loss G: 1.9718, Loss D: -0.2990
Epoch 6, Loss G: 1.8552, Loss D: -0.3927
Epoch 7, Loss G: 1.8639, Loss D: -0.5138
Epoch 8, Loss G: 1.7805, Loss D: -0.6119
Epoch 9, Loss G: 1.8016, Loss D: -0.6829
Epoch 10, Loss G: 1.7766, Loss D: -0.7864


### Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from our model just like we did with the `GaussianCopula` model.

In [11]:
sampled = model.sample(1000)

This will return a table identical to the one which the model was fitted on, but filled with `synthetic` data
which resembles the original one.

In [12]:
sampled.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,73,?,-6473,Some-college,16,Married-spouse-absent,Priv-house-serv,Own-child,Asian-Pac-Islander,Male,84,-9,32,China,<=50K
1,69,Local-gov,98024,12th,9,Separated,Prof-specialty,Wife,Black,Male,4230,5,35,China,<=50K
2,101,Federal-gov,70839,Masters,13,Never-married,Protective-serv,Own-child,White,Male,277,5,47,Germany,<=50K
3,65,Local-gov,224648,9th,8,Never-married,Exec-managerial,Wife,White,Female,132028,-25,33,Haiti,<=50K
4,22,Never-worked,442737,HS-grad,13,Separated,Prof-specialty,Unmarried,White,Male,-361,-23,40,Mexico,<=50K
5,43,Local-gov,231281,Some-college,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,555,2,28,England,<=50K
6,72,Local-gov,124415,11th,17,Married-spouse-absent,Farming-fishing,Unmarried,White,Male,72,15,31,Nicaragua,>50K
7,15,Self-emp-not-inc,465701,Assoc-acdm,16,Separated,Sales,Unmarried,Asian-Pac-Islander,Male,121422,-17,40,?,>50K
8,44,State-gov,103371,Assoc-acdm,9,Married-spouse-absent,Adm-clerical,Husband,Black,Female,49,1746,39,Puerto-Rico,<=50K
9,65,Self-emp-inc,92095,HS-grad,8,Married-civ-spouse,Prof-specialty,Not-in-family,Other,Male,8195,-19,40,Dominican-Republic,<=50K


## Frequently encountered needs

### How can I evaluate the quality of my synthetic data?

In some cases, you will want to know how similar the generated is to the original one.

For this you can use the `evaluation` framework included in SDV by simply importing the
`sdv.evaluation.evaluate` function and calling it passing it both the synthetic and the
real data.

In [13]:
from sdv.evaluation import evaluate

evaluate(sampled, census)

-154.6791796258576